# Amy's Code

Thank you for providing templates to import libraries, import data, rename columns, apply, and inspect.

In [1]:
## all of our imports live here

import pandas as pd
pd.set_option("display.max_columns", None)  # Show all columns
import re ## this lets me do regular expression work.  googel regex for details
import numpy as np
import os
from datetime import date


In [2]:
## import the dataset:

raw = pd.read_csv("../Data/Statistics_Survey2025.csv")

In [3]:
## Pull names!
raw.columns

Index(['Timestamp', 'What is your program?', 'What is your gender?',
       'What is your AGE as of your last birthday?',
       'What is your HEIGHT (to the nearest centimetre)?',
       'What is your WEIGHT (to the nearest kilogram)?',
       'What was your AVERAGE MONTHLY SALARY (to the nearest $100) at your last full time employment prior to enrolling at BCIT?',
       'What MONTHLY SALARY (to the nearest $100) would you EXPECT if you were to seek employment immediately after obtaining your diploma?',
       'After graduation, for which one of the following TYPES OF ORGANIZATION would you most prefer to work?',
       'Do you LIVE WITH YOUR PARENTS? ',
       'What are your MONTHLY EXPENSES for FOOD and LODGING?',
       'What MODE of TRANSPORT do you use most often to get to and from BCIT?',
       'What is your COST per MONTH for TRANSPORTATION to and from BCIT?',
       'What is your COST per MONTH for ENTERTAINMENT?',
       'Are you right-handed, Left-handed or ambidextrous?',

In [4]:
## Renaming Columns
new_names = ['Timestamp', 'program', 'gender',
       'AGE',
       'HEIGHT',
       'WEIGHT',
       'SALARY_OLD',
       'SALARY_GRAD',
       'DESIRE_WORK_ORG',
       'PARENTS',
       'EXPENSES',
       'TRANSPORT',
       'COST_TRANSPORTATION',
       'COST_ENTERTAINMENT',
       'HAND',
       'CELL_PHONE',
       'SMOKE', 
       'foot_size',
       'alcohol',
       'tattoo',
       'sleep',
       'social',
       'homework',
       'work',
       'travel',
       'tuition',
       'coffee']

## Renamed Columns:

In [5]:
## make a new dataframe
Survey = raw

## and rename it
Survey.columns = new_names

## inspect
Survey.columns

Index(['Timestamp', 'program', 'gender', 'AGE', 'HEIGHT', 'WEIGHT',
       'SALARY_OLD', 'SALARY_GRAD', 'DESIRE_WORK_ORG', 'PARENTS', 'EXPENSES',
       'TRANSPORT', 'COST_TRANSPORTATION', 'COST_ENTERTAINMENT', 'HAND',
       'CELL_PHONE', 'SMOKE', 'foot_size', 'alcohol', 'tattoo', 'sleep',
       'social', 'homework', 'work', 'travel', 'tuition', 'coffee'],
      dtype='object')

# My Code

Idea by Samuel, code grammar reviewed and revised by ChatGPT.

## Cleaning

### Cleaning Functions

In [6]:
# Function to distinct column types, ie, numbers should be numbers and strings should be strings

def keep_allnumbers(value):
    """
        inspired by Amy Goldlist, revised by Samuel and ChatGPT
        This function will:
        - Ignores symbols like "$"
        - Preserves parentheses and minus signs
        - Converts:
        - "-$23" → "-23"
        - "($23)" → "(23)"
        - "-($23)" → "-(23)"
    """
    if isinstance(value, (int, float)): 
        return value
    elif isinstance(value, str): 
        match = re.search(r"-?\(\d*\.?\d+\)|-?\d*\.?\d+", value)
        if match:
            extracted = match.group()
            if extracted.startswith("-("):  # Convert "-($23)" → "-(23)"
                return f"-({extracted[2:-1]})"
            return extracted
    return np.nan


### Cleaning Steps

#### 1. General Number Cleaning: Remove symbols, keep +/- and accounting deficit notation i.e. (11)

In [7]:
Cleaned=Survey #create Cleaned df with org data

#Clean org data and store to Cleaned
Cleaned["AGE"] = Survey["AGE"].apply(keep_allnumbers)
Cleaned["HEIGHT"] = Survey["HEIGHT"].apply(keep_allnumbers)
Cleaned["WEIGHT"] = Survey["WEIGHT"].apply(keep_allnumbers)
Cleaned["SALARY_OLD"] = Survey["SALARY_OLD"].apply(keep_allnumbers)
Cleaned["SALARY_GRAD"] = Survey["SALARY_GRAD"].apply(keep_allnumbers)
Cleaned["EXPENSES"] = Survey["EXPENSES"].apply(keep_allnumbers)
Cleaned["COST_TRANSPORTATION"] = Survey["COST_TRANSPORTATION"].apply(keep_allnumbers)
Cleaned["COST_ENTERTAINMENT"] = Survey["COST_ENTERTAINMENT"].apply(keep_allnumbers)
Cleaned["CELL_PHONE"] = Survey["CELL_PHONE"].apply(keep_allnumbers)
Cleaned["foot_size"] = Survey["foot_size"].apply(keep_allnumbers)
Cleaned["alcohol"] = Survey["alcohol"].apply(keep_allnumbers)
Cleaned["sleep"] = Survey["sleep"].apply(keep_allnumbers)
Cleaned["social"] = Survey["social"].apply(keep_allnumbers)
Cleaned["homework"] = Survey["homework"].apply(keep_allnumbers)
Cleaned["work"] = Survey["work"].apply(keep_allnumbers)
Cleaned["coffee"] = Survey["coffee"].apply(keep_allnumbers)


No acccounting notation found in dataset, so no need for further cleaning

#### 2. Custom Clean Category

##### Function

In [8]:
def clean_categories(column, case="upper", mapping=None):
    """
    Cleans categorical data by:
    - Converting to string
    - Stripping whitespace
    - Changing case (default: lowercase)
    - Replacing known variations using a mapping dictionary (if provided)

    Parameters:
    - column (pd.Series): The Pandas column to clean.
    - case (str): "lower" (default), "upper", or "title" for case formatting.
    - mapping (dict, optional): Dictionary to unify known variations.

    Returns:
    - pd.Series: The cleaned column.
    """
    if column is None:
        return None

    # Ensure it's a string and strip spaces
    column = column.astype(str).str.strip()

    # Replace string representations of missing values with NaN
    column = column.replace(["NAN", "nan", "None"], pd.NA)

    # Convert case as needed
    if case == "lower":
        column = column.str.lower()
    elif case == "upper":
        column = column.str.upper()
    elif case == "title":
        column = column.str.title()  # Capitalizes first letter of each word
    elif case == "skip":
        pass

    # Apply mapping if provided
    if mapping:
        column = column.replace(mapping)

    return column

def convert_yes_no(column):
    """
    Converts 'Yes' to True, 'No' to False, and any other value to NaN.

    Parameters:
    - column (pd.Series): The Pandas column to clean.

    Returns:
    - pd.Series: Column with True, False, or NaN.
    """
    return column.astype(str).str.strip().str.lower().map({
        "yes": True,
        "no": False
    })


#### 3. Apply cleaning with category mapping

In [9]:
# Define category mappings
program_mapping = {
    "MKTG": "MKTG",
    "MARKETING": "MKTG",
    "ITMG" : "BITMAN",
    "EXCHANGE" : pd.NA,
    "ITMG (RETAKING COURSES FOR GRADUATION)" : pd.NA,
    "MARKETING MANAGEMENT" : "MKTG",
    "MKTG2J" : "MKTG",
    "MMGT" : "MKTG",
    "MTKG" : "MKTG"
}

#Non-binary will be Nan
gender_mapping = {
    "prefer not to say" : pd.NA,
    "gender apathetic" : pd.NA,
    "non binary" : pd.NA,
    "shakespearian cat" : pd.NA,
    "<na>" : pd.NA
}

transport_mapping = {
    "bus": "public transport",
    "subway": "public transport",
    "bike": "bicycle",
    "walk": "walking"
}

# Apply function to relevant columns
Cleaned["program"] = clean_categories(Cleaned["program"], case="upper", mapping=program_mapping)
Cleaned["gender"] = clean_categories(Cleaned["gender"], case="lower", mapping=gender_mapping)
Cleaned["TRANSPORT"] = clean_categories(Cleaned["TRANSPORT"], mapping=transport_mapping)
Cleaned["DESIRE_WORK_ORG"] = clean_categories(Cleaned["DESIRE_WORK_ORG"])
Cleaned["HAND"] = clean_categories(Cleaned["HAND"])
Cleaned["SMOKE"] = convert_yes_no(Cleaned["SMOKE"])
Cleaned["PARENTS"] = convert_yes_no(Cleaned["PARENTS"])
Cleaned["tattoo"] = convert_yes_no(Cleaned["tattoo"])
Cleaned["travel"] = convert_yes_no(Cleaned["travel"])
Cleaned["tuition"] = convert_yes_no(Cleaned["tuition"])


#### 4. Cleaning Outliers

In [10]:
# Function to clean outliers using IQR method
def clean_outliers(df, columns):
    """
    Removes outliers from specified columns using the Interquartile Range (IQR) method.
    Outliers are defined as values below Q1 - 1.5 * IQR or above Q3 + 1.5 * IQR.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.
        columns (list): List of column names to clean.

    Returns:
        pd.DataFrame: A new DataFrame with outliers replaced with NaN.
    """
    df_cleaned = df.copy()
    
    for col in columns:
        if col in df_cleaned.columns:
            # Convert column to numeric (in case of string numbers)
            df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
            
            # Calculate Q1 (25th percentile) and Q3 (75th percentile)
            Q1 = df_cleaned[col].quantile(0.25)
            Q3 = df_cleaned[col].quantile(0.75)
            IQR = Q3 - Q1
            
            # Define bounds for outliers
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Replace outliers with NaN
            df_cleaned[col] = df_cleaned[col].apply(lambda x: x if lower_bound <= x <= upper_bound else np.nan)
    
    return df_cleaned

# Define columns to clean
columns_to_clean = [
    "AGE", "HEIGHT", "WEIGHT", "SALARY_OLD", "SALARY_GRAD", "EXPENSES",
    "COST_TRANSPORTATION", "COST_ENTERTAINMENT", "CELL_PHONE", "foot_size",
    "alcohol", "sleep", "social", "homework", "work", "coffee"
]

# Apply function to Cleaned dataset
Cleaned = clean_outliers(Cleaned, columns_to_clean)


#### 5. Remove Rows with null

In [11]:
columns_to_clean = ["AGE", "HEIGHT", "WEIGHT", "PARENTS"]
Cleaned = Survey.dropna(subset=columns_to_clean)

### Calculating BMI

In [12]:
# Function to calculate BMI
def calculate_bmi(weight, height):
    """
    Calculates BMI using the formula:
        BMI = weight (kg) / (height (m) ** 2)
    Handles NaN values safely.
    """
    try:
        height_m = float(height) / 100  # Convert cm to meters
        weight_kg = float(weight)
        if height_m > 0 and weight_kg > 0:
            return round(weight_kg / (height_m ** 2), 2)
        else:
            return np.nan  # Return NaN if values are invalid
    except:
        return np.nan  # Handle errors gracefully

# Apply BMI calculation to dataset
Cleaned["BMI"] = Cleaned.apply(lambda row: calculate_bmi(row["WEIGHT"], row["HEIGHT"]), axis=1)

# Inspect the new column
Cleaned[["HEIGHT", "WEIGHT", "BMI"]].head()


C:\Users\samue\AppData\Local\Temp\ipykernel_32260\2631069795.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cleaned["BMI"] = Cleaned.apply(lambda row: calculate_bmi(row["WEIGHT"], row["HEIGHT"]), axis=1)


,HEIGHT,WEIGHT,BMI
0,177.8,110,34.80
1,175,63,20.57
2,182.88,100,29.90
3,5,61.3,24520.00
4,160,54,21.09


### Remove rows that don't have program data

In [13]:
Cleaned = Cleaned.dropna(subset=['program'])

# Export code ideal by Samuel, fulfilled by ChatGPT

In [14]:
# Get the current script's directory (Jupyter-safe method)
script_dir = os.path.abspath(os.path.dirname(os.getcwd()))

# Move up one level from 'Scripts' to 'Baby' and set the correct 'Cleaned Data' path
output_folder = os.path.join(script_dir, "Cleaned Data")

# Ensure the 'Cleaned Data' folder exists
os.makedirs(output_folder, exist_ok=True)

# Define the file name with the full path
file_name = os.path.join(output_folder, "Samuel_" + date.today().isoformat() + ".csv")

# Save the file
Cleaned.to_csv(file_name, index=False)

print(f"File saved to: {file_name}")

File saved to: c:\repos\BA45\Baby\Cleaned Data\Samuel_2025-02-28.csv
